In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#load the training data
train_values = pd.read_csv('/Users/aneruthmohanasundaram/Documents/GitHub/R-Basics/Machine Learning/Datasets/Training_set_values.csv')

#load the training set labels
train_labels = pd.read_csv('/Users/aneruthmohanasundaram/Documents/GitHub/R-Basics/Machine Learning/Datasets/Training_set_labels.csv')

#combine training and the labels for it
combined = train_values.merge(train_labels, on='id')

In [5]:
combined.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [6]:
combined.drop('recorded_by', axis=1, inplace=True)
combined.drop('amount_tsh', axis=1, inplace=True)
combined.drop('public_meeting', axis=1, inplace=True)
combined.drop('permit', axis=1, inplace=True)

## He is changinf the names which are common and renaming it to a values same as the next one 

In [7]:
import warnings
warnings.filterwarnings("ignore")

combined['extraction_type'][combined.extraction_type.isin(['india mark ii','india mark iii'])] = 'india mark'
combined['extraction_type_class'][combined.extraction_type_class.isin(['india mark ii','india mark iii'])] = 'india mark'
combined['extraction_type'][combined.extraction_type.isin(['cemo', 'climax'])] = 'motorpump'
combined['extraction_type_class'][combined.extraction_type_class.isin(['cemo', 'climax'])] = 'motorpump'
combined['extraction_type'][combined.extraction_type.isin(['other - mkulima/shinyanga','other - play pump', 'walimi'])] = 'handpump'
combined['extraction_type_class'][combined.extraction_type_class.isin(['other - mkulima/shinyanga','other - play pump', 'walimi'])] = 'handpump'
combined['extraction_type'][combined.extraction_type.isin(['other - swn 81','swn 80'])] = 'swn'
combined['extraction_type_class'][combined.extraction_type_class.isin(['other - swn 81','swn 80'])] = 'swn'

In [9]:
combined.drop('extraction_type_group', axis=1, inplace=True)
combined.drop('scheme_name', axis=1, inplace=True)
combined.drop('payment', axis=1, inplace=True)
combined.drop('quality_group', axis=1, inplace=True)
combined.drop('quantity_group', axis=1, inplace=True)
combined.drop('source_type', axis=1, inplace=True)
combined.drop('waterpoint_type_group', axis=1, inplace=True)

In [12]:
def get_setting(x):
    x=str(x)
    if x.find('Rural') >= 0: return 'rural'
    if x.find('Urban') >= 0: return 'urban'
    return 'other'
combined['rural_urban'] = combined.lga.apply(get_setting)

In [16]:
type(combined.date_recorded)

pandas.core.series.Series

## Converting the datetime object to datetime64[[ns]] thingy

In [17]:
combined['date_recorded'] = pd.to_datetime(combined['date_recorded'])
combined['construction_year'] = combined.construction_year.map(lambda x: np.nan if x == 0 else x) #. Assigining the 0 values to Nan and converting it
combined['operation_year'] = combined.date_recorded.dt.year - combined.construction_year

In [19]:
combined.date_recorded

0       2011-03-14
1       2013-03-06
2       2013-02-25
3       2013-01-28
4       2011-07-13
           ...    
59395   2013-05-03
59396   2011-05-07
59397   2011-04-11
59398   2011-03-08
59399   2011-03-23
Name: date_recorded, Length: 59400, dtype: datetime64[ns]

In [22]:
combined['operation_year'] = combined.date_recorded.dt.year - combined.construction_year
combined[['operation_year', 'date_recorded', 'construction_year']].head(10)

,operation_year,date_recorded,construction_year
0,12.0,2011-03-14,1999.0
1,3.0,2013-03-06,2010.0
2,4.0,2013-02-25,2009.0
3,27.0,2013-01-28,1986.0
4,NaN,2011-07-13,NaN
5,2.0,2011-03-13,2009.0
6,NaN,2012-10-01,NaN
7,NaN,2012-10-09,NaN
8,NaN,2012-11-03,NaN
9,NaN,2011-08-03,NaN


In [23]:
combined['month_recorded'] = combined.date_recorded.dt.month

In [26]:
def get_season (x):
    if x <= 2: return 1 #short dry season 
    elif x <= 5: return 2 #long rains
    elif x <= 10: return 3 #long dry season
    elif x<= 12: return 4 # short rains
    return
combined['season']=combined.month_recorded.apply(get_season)

# For every categorical response, convert the levels to lower case, in case there is random capitalization

In [29]:
combined.installer = combined.installer.str.lower()
combined.funder = combined.funder.str.lower()
combined.wpt_name = combined.wpt_name.str.lower()
combined.basin = combined.basin.str.lower()
combined.subvillage = combined.subvillage.str.lower()
combined.region = combined.region.str.lower()
combined.lga = combined.lga.str.lower()
combined.ward = combined.ward.str.lower()
combined.management = combined.management.str.lower()
combined.management_group = combined.management_group.str.lower()
combined.scheme_management = combined.scheme_management.str.lower()

# probably not the most elegant but i'm filling NaN with zeros to impute mean coordinates, population, and construction year where missing

In [30]:
combined['longitude'].replace('0', np.nan, inplace=True)
combined['latitude'].replace('0', np.nan, inplace=True)
combined['gps_height'].replace('0', np.nan, inplace=True)
combined['construction_year'].replace('0', np.nan, inplace=True)
combined['operation_year'].replace('0', np.nan, inplace=True)
combined['population'].replace('0', np.nan, inplace=True)

# impute mean for location coordinates, year, and population using groupby for region and district when the district code info isn't available I impute just the regional mean

In [31]:
combined["latitude"].fillna(combined.groupby(['region', 'district_code'])["latitude"].transform("mean"), inplace=True)
combined["longitude"].fillna(combined.groupby(['region', 'district_code'])["longitude"].transform("mean"), inplace=True)
combined["longitude"].fillna(combined.groupby(['region'])["longitude"].transform("mean"), inplace=True)
combined["gps_height"].fillna(combined.groupby(['region', 'district_code'])["gps_height"].transform("mean"), inplace=True)
combined["gps_height"].fillna(combined.groupby(['region'])["gps_height"].transform("mean"), inplace=True)
combined["gps_height"].fillna(combined["gps_height"].mean(), inplace=True)
combined["construction_year"].fillna(combined["construction_year"].mean(), inplace=True)
combined["population"].fillna(combined.groupby(['region', 'district_code'])["population"].transform("mean"), inplace=True)
combined["population"].fillna(combined.groupby(['region'])["population"].transform("mean"), inplace=True)
combined["population"].fillna(combined["population"].mean(), inplace=True)

# Replacing Nan with other in funder,installer,scheme_management,subvillage

In [32]:
combined["funder"].fillna("other", inplace=True)
combined["installer"].fillna("other", inplace=True)
combined["scheme_management"].fillna("other", inplace=True)
combined["subvillage"].fillna("other", inplace=True)

# recalculate operation year

In [34]:
combined['operation_year'] = combined.date_recorded.dt.year - combined.construction_year

In [35]:
combined['operation_year']

0        12.000000
1         3.000000
2         4.000000
3        27.000000
4        14.185314
           ...    
59395    14.000000
59396    15.000000
59397    14.185314
59398    14.185314
59399     9.000000
Name: operation_year, Length: 59400, dtype: float64

# Exporting the dataset

In [ ]:
# pd.DataFrame(combined).to_csv("cleaned_v2.csv")

# Reapeating the same for test dataset